In [ ]:
%pip install huggingface_hub
%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
%pip install transformers
%pip install datasets
%pip install evaluate

from huggingface_hub import snapshot_download, login
import os

token = ""
#Your HF token.
login(token=token)

local_dir = snapshot_download("google/gemma-3-12b-it")
print("Files are in:", local_dir)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, copy

def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

model_path = "/root/.cache/huggingface/hub/models--google--gemma-3-12b-it/snapshots/96b6f1eccf38110c56df3a15bffe176da04bfd80"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda").eval()
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_path)

new_layer = copy.deepcopy(model.model.language_model.layers[0]).to("cuda").eval()
#new_layer = copy.deepcopy(model.model.language_model.layers[23]).to("cuda").eval()
#new_layer = copy.deepcopy(model.model.language_model.layers[47]).to("cuda").eval()

'''
Zeroed out
'''
#excluded = set(new_layer.post_attention_layernorm.parameters()) | set(new_layer.post_feedforward_layernorm.parameters())
#
#with torch.no_grad():
#    for p in new_layer.parameters():
#        if p not in excluded:
#            p.data.zero_()

'''
Randomly re-initialized
'''
#def reset_weights(m):
#    if hasattr(m, 'reset_parameters'):
#        m.reset_parameters()
#with torch.no_grad():
#    new_layer.apply(reset_weights)

'''
Briefly re-trained
'''
#ckpt_path = "retrained_layer_0.pt"
#ckpt_path = "retrained_layer_23.pt"
#with torch.no_grad():
#    state_dict = torch.load(ckpt_path, map_location="cuda")["model_state_dict"]
#    new_layer.load_state_dict(state_dict, strict=True)

with torch.no_grad():
    model.model.language_model.layers[0] = new_layer
#    model.model.language_model.layers[23] = new_layer
#    model.model.language_model.layers[47] = new_layer

In [ ]:
prompt = "Tell a story:"
#prompt = "Factor x^2+3x+2:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
gen_ids = model.generate(
    input_ids,
    max_new_tokens=50,
    do_sample=True,
    temperature=1.0,
    use_cache=True
)

print(tokenizer.decode(gen_ids[0], skip_special_tokens=True))

In [ ]:
import evaluate
from datasets import load_dataset
import math

ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
#ds = load_dataset("m-ric/huggingface_doc", split="train")
texts = ["\n".join(ds["text"])]

ppl = evaluate.load("perplexity", module_type="measurement")

save_dir = "model_temp_save"
model.save_pretrained(save_dir); tokenizer.save_pretrained(save_dir)

with torch.no_grad():
    res = ppl.compute(model_id=save_dir, data=texts, add_start_token=True, batch_size=8, max_length=512)

print("PPL:", res["mean_perplexity"])